# Finding a functional roll in a slding window across a BGC

## 1. Datafiles required
In this tutorial we will use the following file:  
 I.`mibig_prot_seqs_2.0.fasta` a fasta file of the MIBiG database.    
 II.`antibiotics-metadata.tsv` a tab-separated file with MIBiG metadata.    
 III.`Rast-ids.tsv` a tab separated file with Rast Ids.  
 
 
- 1.1 Lets see the first lines of the file `mibig_prot_seqs_2.0.fasta`

In [ ]:
%%capture output
# Install a conda package in the current Jupyter kernel

import sys
!conda install --yes --prefix {sys.prefix} numpy;
!conda install --yes --prefix {sys.prefix} pandas; 
!conda install --yes --prefix {sys.prefix} blast;
!conda install --yes --prefix {sys.prefix} matplotlib;
#https://jakevdp.github.io/blog/2017/12/05/installing-python-packages-from-jupyter/


In [ ]:
%%bash
head -n4 mibig_prot_seqs_2.0.fasta

- 1.2 Lets see the first lines of the file `antibiotics-metadata.tsv`

In [ ]:
%%bash
head -n4  antibiotics-metadata.tsv  

**This table contian genome Id when available** Notice that column three contains genome NCBI Id in the case that the species that produces this BGC has a complete genome available at NCBI.

In [ ]:
%%bash
cut -f 3 antibiotics-metadata.tsv  | sort| uniq | head -n3



- 1.3 Lets see the first lines of the file `Rast-ids.tsv`

In [ ]:
%%bash 
head -n4 Rast-ids.tsv

**Rast Job Id** This tab separated file contains three columns, the first one is the RAST Job Id, second one RAST genome Id and finally the  third column contains the organisms name.

##. Blast databases

## 3. Obtaining BGC borders
First we will run `getSeq.sh` to obtain the sequences at the borders of a each BGC.  
`bash getSeq <BGC> <MiBiG fasta file>`  

- 1 BGCId 
- 2 MiBiG fasta db (default mibig_prot_seqs_2.0.fasta)
- 3 MiBig metadata including in third column genome Id if exist (antibiotics-metadata.tsv)
- 4 File with Rast Ids (Rast-ids.tsv)
- 5 Sliding Window size (10 in this tutorial)
- 6 Number of windows (5 in this tutorial) 
- 7 Functional word to search in genome functional annotation file (ansporter that stands for [Tt]ransporters  

This script needs as input a BGC Id from MIBiG and the MIBiG fasta file, for example `BGC0000406`.      
- getSeq obtains the fasta files `BGC0000406.i` and `BGC0000406.f`   that stands for initial and final sequence of the BGC.  
- Identifies the genome that contains this BGC using the metadata file.  
- Run a Blast search and finds the best hit of each one of the border sequences. 
- If they correspond to the BGC, call the perl script `getTrans.pl` to serach in the annotation file genes taht corresponds to the functional category in a sliding window.
  src/getTrans.pl 6772 6744 220639 BGC0000406 10 5 ansporter




2.1 First lets see that at the moment, there are not files with extensions .i or .f

In [ ]:
ls *i *f

2.2 Now lets run getSeq

In [ ]:
%%capture output
%%bash src/getSeq.sh BGC0000406 mibig_prot_seqs_2.0.fasta antibiotics-metadata.tsv Rast-ids.tsv 10 5 ansporter
    

In [ ]:
%%bash
echo BGC$'\t'0$'\t'1$'\t'2$'\t'3$'\t'4$'\t'5 > bgc-data.out
cat bgc-data.out

In [ ]:
  %%capture output
  %%bash 
   chmod +x src/getSeq.sh

    cat sampleBGC.txt | while read line
    do
   src/getSeq.sh $line mibig_prot_seqs_2.0.fasta antibiotics-metadata.tsv Rast-ids.tsv 10 5 ansporter
    done | perl -ne 'print if ($. % 3==0)' | grep -v not >> bgc-data.out
    

In [ ]:
%%bash
cat bgc-data.out

2.3 Find the genome file that corresponds to this BGC
BGC BGC0000431 genome ABYB01 file 512417

2.4 Find the corresponding border gene in that genome
getTrans.pl 635 656 512417 BGC0000431


2.5 Look in a window if there are transporters
BGC0000406	0	0	0	0	0

In [ ]:
import pandas as pd
data = pd.read_csv('bgc-data.out',sep='\t',index_col='BGC')
print(data)

df=data.transpose()
#print(df)
#print(df.iloc[0])



In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

windowNumber = [0, 1, 2, 3,4,5]
transporterGenes = [0,1,1,1,4,6]

plt.scatter(windowNumber, transporterGenes)
plt.xlabel('Window Number (10 genes each window)')
plt.ylabel('Number of Genes classified as transporter related ')



In [ ]:
data.boxplot(column=['0','1', '2', '3', '4', '5'])
#data.plot.scatter(x='1', y='2', title= "Scatter plot between two columns of a multi-column DataFrame")

In [ ]:
# data.plot.scatter(column=['0','1', '2', '3', '4', '5'])
#data.plot.scatter(x='1', y='2', title= "Scatter plot between two columns of a multi-column DataFrame")
df.plot(kind='bar', stacked=True);
#https://pandas.pydata.org/pandas-docs/version/0.9.1/visualization.html

In [ ]:
#!conda install --yes --prefix {sys.prefix} pandas.tools;
#from pandas.tools.plotting import radviz
#plt.figure()
#radviz(data, 'BGC')

In [ ]:
from matplotlib import cm
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# initialize dataframe
n = 200
ngroup = 3
df = pd.DataFrame({'data': np.random.rand(n), 'group': map(np.floor, np.random.rand(n) * ngroup)})

group = 'group'
column = 'data'
grouped = df.groupby(group)

names, vals, xs = [], [] ,[]

for i, (name, subdf) in enumerate(grouped):
    names.append(name)
    vals.append(subdf[column].tolist())
    xs.append(np.random.normal(i+1, 0.04, subdf.shape[0]))

plt.boxplot(vals, labels=names)
ngroup = len(vals)
clevels = np.linspace(0., 1., ngroup)

for x, val, clevel in zip(xs, vals, clevels):
    plt.scatter(x, val, c=cm.prism(clevel), alpha=0.4)